## IO编程
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431917590955542f9ac5f5c1479faf787ff2b028ab47000

IO在计算机中指Input/Output，也就是输入和输出。由于程序和运行时数据是在内存中驻留，由CPU这个超快的计算核心来执行，涉及到数据交换的地方，通常是磁盘、网络等，就需要IO接口。

比如你打开浏览器，访问新浪首页，浏览器这个程序就需要通过网络IO获取新浪的网页。浏览器首先会发送数据给新浪服务器，告诉它我想要首页的HTML，这个动作是往外发数据，叫Output，随后新浪服务器把网页发过来，这个动作是从外面接收数据，叫Input。所以，通常，程序完成IO操作会有Input和Output两个数据流。当然也有只用一个的情况，比如，从磁盘读取文件到内存，就只有Input操作，反过来，把数据写到磁盘文件里，就只是一个Output操作。

IO编程中，Stream（流）是一个很重要的概念，可以把流想象成一个水管，数据就是水管里的水，但是只能单向流动。Input Stream就是数据从外面（磁盘、网络）流进内存，Output Stream就是数据从内存流到外面去。对于浏览网页来说，浏览器和新浪服务器之间至少需要建立两根水管，才可以既能发数据，又能收数据。

由于CPU和内存的速度远远高于外设的速度，所以，在IO编程中，就存在速度严重不匹配的问题。举个例子来说，比如要把100M的数据写入磁盘，CPU输出100M的数据只需要0.01秒，可是磁盘要接收这100M数据可能需要10秒，怎么办呢？有两种办法：

第一种是CPU等着，也就是程序暂停执行后续代码，等100M的数据在10秒后写入磁盘，再接着往下执行，这种模式称为同步IO；

另一种方法是CPU不等待，只是告诉磁盘，“您老慢慢写，不着急，我接着干别的事去了”，于是，后续代码可以立刻接着执行，这种模式称为异步IO。

同步和异步的区别就在于是否等待IO执行的结果。好比你去麦当劳点餐，你说“来个汉堡”，服务员告诉你，对不起，汉堡要现做，需要等5分钟，于是你站在收银台前面等了5分钟，拿到汉堡再去逛商场，这是同步IO。

你说“来个汉堡”，服务员告诉你，汉堡需要等5分钟，你可以先去逛商场，等做好了，我们再通知你，这样你可以立刻去干别的事情（逛商场），这是异步IO。

很明显，使用异步IO来编写程序性能会远远高于同步IO，但是异步IO的缺点是编程模型复杂。想想看，你得知道什么时候通知你“汉堡做好了”，而通知你的方法也各不相同。如果是服务员跑过来找到你，这是回调模式，如果服务员发短信通知你，你就得不停地检查手机，这是轮询模式。总之，异步IO的复杂度远远高于同步IO。

## 异步IO

在IO编程一节中，我们已经知道，CPU的速度远远快于磁盘、网络等IO。在一个线程中，CPU执行代码的速度极快，然而，一旦遇到IO操作，如读写文件、发送网络数据时，就需要等待IO操作完成，才能继续进行下一步操作。这种情况称为同步IO。

在IO操作的过程中，当前线程被挂起，而其他需要CPU执行的代码就无法被当前线程执行了。

因为一个IO操作就阻塞了当前线程，导致其他代码无法执行，所以我们必须使用多线程或者多进程来并发执行代码，为多个用户服务。每个用户都会分配一个线程，如果遇到IO导致线程被挂起，其他用户的线程不受影响。

多线程和多进程的模型虽然解决了并发问题，但是系统不能无上限地增加线程。由于系统切换线程的开销也很大，所以，一旦线程数量过多，CPU的时间就花在线程切换上了，真正运行代码的时间就少了，结果导致性能严重下降。

由于我们要解决的问题是CPU高速执行能力和IO设备的龟速严重不匹配，多线程和多进程只是解决这一问题的一种方法。

另一种解决IO问题的方法是异步IO。当代码需要执行一个耗时的IO操作时，它只发出IO指令，并不等待IO结果，然后就去执行其他代码了。一段时间后，当IO返回结果时，再通知CPU进行处理。

可以想象如果按普通顺序写出的代码实际上是没法完成异步IO的：

```
do_some_code()
f = open('/path/to/file', 'r')
r = f.read() # <== 线程停在此处等待IO操作结果
# IO操作完成后线程才能继续执行:
do_some_code(r)
```

所以，同步IO模型的代码是无法实现异步IO模型的。

异步IO模型需要一个消息循环，在消息循环中，主线程不断地重复“读取消息-处理消息”这一过程：

```
loop = get_event_loop()
while True:
    event = loop.get_event()
    process_event(event)
```
消息模型其实早在应用在桌面应用程序中了。一个GUI程序的主线程就负责不停地读取消息并处理消息。所有的键盘、鼠标等消息都被发送到GUI程序的消息队列中，然后由GUI程序的主线程处理。

由于GUI线程处理键盘、鼠标等消息的速度非常快，所以用户感觉不到延迟。某些时候，GUI线程在一个消息处理的过程中遇到问题导致一次消息处理时间过长，此时，用户会感觉到整个GUI程序停止响应了，敲键盘、点鼠标都没有反应。这种情况说明在消息模型中，处理一个消息必须非常迅速，否则，主线程将无法及时处理消息队列中的其他消息，导致程序看上去停止响应。

消息模型是如何解决同步IO必须等待IO操作这一问题的呢？当遇到IO操作时，代码只负责发出IO请求，不等待IO结果，然后直接结束本轮消息处理，进入下一轮消息处理过程。当IO操作完成后，将收到一条“IO完成”的消息，处理该消息时就可以直接获取IO操作结果。

在“发出IO请求”到收到“IO完成”的这段时间里，同步IO模型下，主线程只能挂起，但异步IO模型下，主线程并没有休息，而是在消息循环中继续处理其他消息。这样，在异步IO模型下，一个线程就可以同时处理多个IO请求，并且没有切换线程的操作。对于大多数IO密集型的应用程序，使用异步IO将大大提升系统的多任务处理能力。


## 协程

https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432090171191d05dae6e129940518d1d6cf6eeaaa969000

在学习异步IO模型前，我们先来了解协程。

- 协程，又称微线程，纤程。英文名Coroutine。

协程的概念很早就提出来了，但直到最近几年才在某些语言（如Lua）中得到广泛应用。

- 子程序，或者称为函数，在所有语言中都是层级调用，比如A调用B，B在执行过程中又调用了C，C执行完毕返回，B执行完毕返回，最后是A执行完毕。

所以子程序调用是通过栈实现的，一个线程就是执行一个子程序。

子程序调用总是一个入口，一次返回，调用顺序是明确的。而协程的调用和子程序不同。

- 协程看上去也是子程序，但执行过程中，在子程序内部可中断，然后转而执行别的子程序，在适当的时候再返回来接着执行。

注意，在一个子程序中中断，去执行其他子程序，不是函数调用，有点类似CPU的中断。比如子程序A、B：

```
def A():
    print('1')
    print('2')
    print('3')

def B():
    print('x')
    print('y')
    print('z')
```
假设由协程执行，在执行A的过程中，可以随时中断，去执行B，B也可能在执行过程中中断再去执行A，结果可能是：

```
1
2
x
y
3
z
```

但是在A中是没有调用B的，所以协程的调用比函数调用理解起来要难一些。

看起来A、B的执行有点像多线程，但协程的特点在于是一个线程执行，那和多线程比，协程有何优势？

最大的优势就是协程极高的执行效率。因为子程序切换不是线程切换，而是由程序自身控制，因此，没有线程切换的开销，和多线程比，线程数量越多，协程的性能优势就越明显。

第二大优势就是不需要多线程的锁机制，因为只有一个线程，也不存在同时写变量冲突，在协程中控制共享资源不加锁，只需要判断状态就好了，所以执行效率比多线程高很多。

因为协程是一个线程执行，那怎么利用多核CPU呢？最简单的方法是多进程+协程，既充分利用多核，又充分发挥协程的高效率，可获得极高的性能。

Python对协程的支持是通过generator实现的。

在generator中，我们不但可以通过for循环来迭代，还可以不断调用next()函数获取由yield语句返回的下一个值。

但是Python的yield不但可以返回一个值，它还可以接收调用者发出的参数。


传统的生产者-消费者模型是一个线程写消息，一个线程取消息，通过锁机制控制队列和等待，但一不小心就可能死锁。如果改用协程，生产者生产消息后，直接通过yield跳转到消费者开始执行，待消费者执行完毕后，切换回生产者继续生产，效率极高：

通过g.send(None)或者next(g)启动生成器函数，并执行到第一个yield语句结束的位置。

In [4]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] consuming {}...'.format(n))
        r = '200 OK'
        
def produce(c):
    c.send(None)
    n = 0
    while n<5:
        n = n + 1
        print('[PRODUCE] Producing {}...'.format(n))
        r = c.send(n)
        print('[PRODUCE] Consumer return:{}'.format(r))
    c.close()
    
c = consumer()
produce(c)

[PRODUCE] Producing 1...
[CONSUMER] consuming 1...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 2...
[CONSUMER] consuming 2...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 3...
[CONSUMER] consuming 3...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 4...
[CONSUMER] consuming 4...
[PRODUCE] Consumer return:200 OK
[PRODUCE] Producing 5...
[CONSUMER] consuming 5...
[PRODUCE] Consumer return:200 OK


注意到consumer函数是一个generator，把一个consumer传入produce后：

首先调用c.send(None)启动生成器；

然后，一旦生产了东西，通过c.send(n)切换到consumer执行；

consumer通过yield拿到消息，处理，又通过yield把结果传回；

produce拿到consumer处理的结果，继续生产下一条消息；

produce决定不生产了，通过c.close()关闭consumer，整个过程结束。

整个流程无锁，由一个线程执行，produce和consumer协作完成任务，所以称为“协程”，而非线程的抢占式多任务。

最后套用Donald Knuth的一句话总结协程的特点：

“子程序就是协程的一种特例。”

## asyncio
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432090954004980bd351f2cd4cc18c9e6c06d855c498000

asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。

asyncio的编程模型就是一个消息循环。我们从asyncio模块中直接获取一个EventLoop的引用，然后把需要执行的协程扔到EventLoop中执行，就实现了异步IO。

用asyncio实现Hello world代码如下：

In [1]:
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
loop.close()

Hello world!
Hello again!


@asyncio.coroutine把一个generator标记为coroutine类型，然后，我们就把这个coroutine扔到EventLoop中执行。

hello()会首先打印出Hello world!，然后，yield from语法可以让我们方便地调用另一个generator。由于asyncio.sleep()也是一个coroutine，所以线程不会等待asyncio.sleep()，而是直接中断并执行下一个消息循环。当asyncio.sleep()返回时，线程就可以从yield from拿到返回值（此处是None），然后接着执行下一行语句。

把asyncio.sleep(1)看成是一个耗时1秒的IO操作，在此期间，主线程并未等待，而是去执行EventLoop中其他可以执行的coroutine了，因此可以实现并发执行。

我们用Task封装两个coroutine试试：

In [1]:
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! {}'.format(threading.currentThread()))
    yield from asyncio.sleep(1)
    print('Hello again! {}'.format(threading.currentThread()))
    
loop = asyncio.get_event_loop()
tasks = [hello(),hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world! <_MainThread(MainThread, started 9020)>
Hello world! <_MainThread(MainThread, started 9020)>
Hello again! <_MainThread(MainThread, started 9020)>
Hello again! <_MainThread(MainThread, started 9020)>


由打印的当前线程名称可以看出，两个coroutine是由同一个线程并发执行的。

如果把asyncio.sleep()换成真正的IO操作，则多个coroutine就可以由一个线程并发执行。

## python异步编程之asyncio（百万并发）
https://www.cnblogs.com/shenh/p/9090586.html

### asyncio

下面通过举例来对比同步代码和异步代码编写方面的差异，其次看下两者性能上的差距，我们使用sleep(1)模拟耗时1秒的io操作

#### 同步

输出结果间隔约是1s

In [2]:
import time

def hello():
    time.sleep(1)
    print('Hello World {}'.format(time.time()))
    
def run():
    for i in range(5):
        hello()

if __name__ == '__main__':
    run()

Hello World 1554695117.795327
Hello World 1554695118.7953672
Hello World 1554695119.7954051
Hello World 1554695120.7963617
Hello World 1554695121.7973413


#### 异步


In [3]:
import time
import asyncio


async def hello():
    asyncio.sleep(1)
    print('Hello World {}'.format(time.time()))


def run():
    for i in range(5):
        loop.run_until_complete(hello())


loop = asyncio.get_event_loop()
if __name__ == '__main__':
    run()

Hello World 1554695545.3713374
Hello World 1554695545.3713374
Hello World 1554695545.3713374
Hello World 1554695545.3713374
Hello World 1554695545.3713374


async def 用来定义异步函数，其内部有异步操作。每个线程有一个事件循环，主线程调用asyncio.get_event_loop()时会创建事件循环，你需要把异步的任务丢给这个循环的run_until_complete()方法，事件循环会安排协同程序的执行。

### aiohttp

如果需要并发http请求怎么办呢，通常是用requests，但requests是同步的库，如果想异步的话需要引入aiohttp。这里引入一个类，from aiohttp import ClientSession，首先要建立一个session对象，然后用session对象去打开网页。session可以进行多项操作，比如post, get, put, head等。

## Python异步IO之协程(一):从yield from到async的使用
https://blog.csdn.net/SL_World/article/details/86597738

### 为什么需要协程？

【同步】：就是发出一个“调用”时，在没有得到结果之前，该“调用”就不返回，“调用者”需要一直等待该“调用”结束，才能进行下一步工作。
【异步】：“调用”在发出之后，就直接返回了，也就没有返回结果。“被调用者”完成任务后，通过状态来通知“调用者”继续回来处理该“调用”。

下面我们先来看一个用普通同步代码实现多个IO任务的案例。

In [2]:
import time

def taskIO_1():
    print('开始运行IO任务1...')
    time.sleep(2)
    print('IO任务1已完成，耗时2秒...')
    
def taskIO_2():
    print('开始运行IO任务2...')
    time.sleep(3)
    print('IO任务2已完成，耗时3秒...')
    
start = time.time()
taskIO_1()
taskIO_2()
print('所有IO任务总耗时%.5f秒' % float(time.time()-start))

开始运行IO任务1...
IO任务1已完成，耗时2秒...
开始运行IO任务2...
IO任务2已完成，耗时3秒...
所有IO任务总耗时5.00209秒


上面，我们顺序实现了两个同步IO任务taskIO_1()和taskIO_2()，则最后总耗时就是5秒。我们都知道，在计算机中CPU的运算速率要远远大于IO速率，而当CPU运算完毕后，如果再要闲置很长时间去等待IO任务完成才能进行下一个任务的计算，这样的任务执行效率很低。

所以我们需要有一种异步的方式来处理类似上述任务，会极大增加效率(当然就是协程啦～)。而在Python中生成器中的关键字yield可以实现中断功能。所以起初，协程是基于生成器的变形进行实现的，之后虽然编码形式有变化，但基本原理还是一样的。

### 使用yield from和@asyncio.coroutine实现协程

在Python3.4中，协程都是通过使用yield from和asyncio模块中的@asyncio.coroutine来实现的。asyncio专门被用来实现异步IO操作。

#### 什么是yield from?和yield有什么区别?

1> 我们都知道，yield在生成器中有中断的功能，可以传出值，也可以从函数外部接收值，而yield from的实现就是**简化了yield操作**。让我们先来看一个案例：

In [6]:
def generator_1(titles):
    yield titles
def generator_2(titles):
    yield from titles

titles = ['Python','Java','C++']
for title in generator_1(titles):
    print('生成器1:',title)
for title in generator_2(titles):
    print('生成器2:',title)

生成器1: ['Python', 'Java', 'C++']
生成器2: Python
生成器2: Java
生成器2: C++


在这个例子中yield titles返回了titles完整列表，而yield from titles实际等价于：

```
for title in titles:　
    yield title　
```
2> 而yield from功能还不止于此，它还有一个主要的功能是省去了很多异常的处理，不再需要我们手动编写，其**内部已经实现大部分异常处理。**

举个例子：下面通过生成器来实现一个整数加和的程序，通过send()函数向生成器中传入要加和的数字，然后最后以返回None结束，total保存最后加和的总数。

In [2]:
def generator_1():  # 子生成器
    total = 0
    while True:
        x = yield
        print('加',x)
        if not x:
            break
        total += x
    return total

def generator_2(): # 委托生成器
    while True:
        total = yield from generator_1()
        print('加和总数是:',total)
        
def main(): # 调用方
    g1 = generator_1()
    g1.send(None)
    g1.send(2)
    g1.send(3)
    g1.send(None)

main()

加 2
加 3
加 None


StopIteration: 5

可见对于生成器g1，在最后传入None后，程序退出，报StopIteration异常并返回了最后total值是5。如果使用委托生成器，则结果如下。可见yield from封装了处理常见异常的代码。对于g2即便传入None也不报异常，其中total = yield from generator_1()返回给total的值是generator_1()最终的return total

In [3]:
def generator_1():  # 子生成器
    total = 0
    while True:
        x = yield
        print('加',x)
        if not x:
            break
        total += x
    return total

def generator_2(): # 委托生成器
    while True:
        total = yield from generator_1()
        print('加和总数是:',total)
        
def main(): # 调用方
    g2 = generator_2()
    g2.send(None)
    g2.send(2)
    g2.send(3)
    g2.send(None)

main()

加 2
加 3
加 None
加和总数是: 5


对于g2即便传入None也不报异常，其中total = yield from generator_1()返回给total的值是generator_1()最终的return total

借用上述例子，这里有几个概念需要理一下：

【子生成器】：yield from后的generator_1()生成器函数是子生成器

【委托生成器】：generator_2()是程序中的委托生成器，它负责委托子生成器完成具体任务

【调用方】：main()是程序中的调用方，负责调用委托生成器

3>**yield from在其中还有一个关键的作用是：建立调用方和子生成器的通道**

在上述代码中main()每一次在调用send(value)时，value不是传递给了委托生成器generator_2()，而是借助yield from传递给了子生成器generator_1()中的yield

同理，子生成器中的数据也是通过yield直接发送到调用方main()中。之后我们的代码都依据调用方-子生成器-委托生成器的规范形式书写。

#### 如何结合@asyncio.coroutine实现协程

那yield from通常用在什么地方呢？在协程中，只要是和IO任务类似的、耗费时间的任务都需要使用yield from来进行中断，达到异步功能！
我们在上面那个同步IO任务的代码中修改成协程的用法如下：

In [1]:
# 使用同步方式编写异步功能
import time
import asyncio

@asyncio.coroutine # 标志协程的装饰器
def taskIO_1():
    print('开始运行IO任务1...')
    yield from asyncio.sleep(2) 
    print('IO任务1已完成，耗时2秒...')
    return taskIO_1.__name__

@asyncio.coroutine # 标志协程的装饰器    
def taskIO_2():
    print('开始运行IO任务2...')
    yield from asyncio.sleep(3)
    print('IO任务2已完成，耗时3秒...')
    return taskIO_2.__name__

@asyncio.coroutine # 标志协程的装饰器   
def main():  # 调用方
    tasks = [taskIO_1(),taskIO_2()]  # 把所有任务添加到task中
    done,pending = yield from asyncio.wait(tasks)  # 子生成器
    for r in done:  # done和pending都是一个任务，所以返回结果需要逐个调用result()
        print('协程无序返回值：'+r.result())
    
if __name__ == '__main__':
    start = time.time()
    loop = asyncio.get_event_loop()  # 创建一个事件循环对象loop
    try:
        loop.run_until_complete(main()) # 完成事件循环，直到最后一个任务结束
    finally:
        loop.close()
    print('所有IO任务总耗时%.5f秒' % float(time.time()-start))

开始运行IO任务2...
开始运行IO任务1...
IO任务1已完成，耗时2秒...
IO任务2已完成，耗时3秒...
协程无序返回值：taskIO_1
协程无序返回值：taskIO_2
所有IO任务总耗时3.00852秒


【使用方法】： @asyncio.coroutine装饰器是协程函数的标志，我们需要在每一个任务函数前加这个装饰器，并在函数中使用yield from。在同步IO任务的代码中使用的time.sleep(2)来假设任务执行了2秒。但在协程中yield from后面必须是子生成器函数，而time.sleep()并不是生成器，所以这里需要使用内置模块提供的生成器函数asyncio.sleep()。
【功能】：通过使用协程，极大增加了多任务执行效率，最后消耗的时间是任务队列中耗时最多的时间。上述例子中的总耗时3秒就是taskIO_2()的耗时时间。
【执行过程】：
1. 上面代码先通过get_event_loop()获取了一个标准事件循环loop(因为是一个，所以协程是单线程)
2. 然后，我们通过run_until_complete(main())来运行协程(此处把调用方协程main()作为参数，调用方负责调用其他委托生成器)，run_until_complete的特点就像该函数的名字，直到循环事件的所有事件都处理完才能完整结束。
3. 进入调用方协程，我们把多个任务[taskIO_1()和taskIO_2()]放到一个task列表中，可理解为打包任务。
4. 现在，我们使用asyncio.wait(tasks)来获取一个awaitable objects即可等待对象的集合(此处的aws是协程的列表)，并发运行传入的aws，同时通过yield from返回一个包含(done, pending)的元组，done表示已完成的任务列表，pending表示未完成的任务列表，欲取已完成任务的结果，需遍历其列表的每个任务并逐个调用result()方法；如果使用asyncio.as_completed(tasks)则会按完成顺序生成协程的迭代器(常用于for循环中)，因此当你用它迭代时，会尽快得到每个可用的结果。
```
for task in asyncio.as_completed(tasks):
        resualt = await task
        print(resualt)
```
【此外，当轮询到某个事件时(如taskIO_1())，直到遇到该任务中的yield from中断，开始处理下一个事件(如taskIO_2()))，当yield from后面的子生成器完成任务时，该事件才再次被唤醒】
5. 因为done里面有我们需要的返回结果，但它目前还是个任务列表，所以要取出返回的结果值，我们遍历它并逐个调用result()取出结果即可。(注：对于asyncio.wait()和asyncio.as_completed()返回的结果均是先完成的任务结果排在前面，所以此时打印出的结果不一定和原始顺序相同，但使用gather()的话可以得到原始顺序的结果集，两者更详细的案例说明见此)
6. 最后我们通过loop.close()关闭事件循环。

综上所述：协程的完整实现是靠①事件循环＋②协程。

### 使用async和await实现协程

在Python 3.4中，我们发现很容易将协程和生成器混淆(虽然协程底层就是用生成器实现的)，所以在后期加入了其他标识来区别协程和生成器。

在Python 3.5开始引入了新的语法async和await，以简化并更好地标识异步IO。

要使用新的语法，只需要做两步简单的替换：

1. 把@asyncio.coroutine替换为async；

2. 把yield from替换为await。

In [1]:
# 使用同步方式编写异步功能
import time
import asyncio


async def taskIO_1():
    print('开始运行IO任务1...')
    await asyncio.sleep(2) 
    print('IO任务1已完成，耗时2秒...')
    return taskIO_1.__name__


async def taskIO_2():
    print('开始运行IO任务2...')
    await asyncio.sleep(3)
    print('IO任务2已完成，耗时3秒...')
    return taskIO_2.__name__


async def main():  # 调用方
    tasks = [taskIO_1(),taskIO_2()]  # 把所有任务添加到task中
    done,pending = await asyncio.wait(tasks)  # 子生成器
    for r in done:  # done和pending都是一个任务，所以返回结果需要逐个调用result()
        print('协程无序返回值：'+r.result())
    
if __name__ == '__main__':
    start = time.time()
    loop = asyncio.get_event_loop()  # 创建一个事件循环对象loop
    try:
        loop.run_until_complete(main()) # 完成事件循环，直到最后一个任务结束
    finally:
        loop.close()
    print('所有IO任务总耗时%.5f秒' % float(time.time()-start))

开始运行IO任务2...
开始运行IO任务1...
IO任务1已完成，耗时2秒...
IO任务2已完成，耗时3秒...
协程无序返回值：taskIO_2
协程无序返回值：taskIO_1
所有IO任务总耗时3.00637秒


### 总结

最后我们将整个过程串一遍。

【引出问题】：

同步编程的并发性不高
多进程编程效率受CPU核数限制，当任务数量远大于CPU核数时，执行效率会降低。
多线程编程需要线程之间的通信，而且需要锁机制来防止共享变量被不同线程乱改，而且由于Python中的GIL(全局解释器锁)，所以实际上也无法做到真正的并行。

【产生需求】：

可不可以采用同步的方式来编写异步功能代码？
能不能只用一个单线程就能做到不同任务间的切换？这样就没有了线程切换的时间消耗，也不用使用锁机制来削弱多任务并发效率！
对于IO密集型任务，可否有更高的处理方式来节省CPU等待时间？

【结果】：

所以协程应运而生。当然，实现协程还有其他方式和函数，以上仅展示了一种较为常见的实现方式。此外，多进程和多线程是内核级别的程序，而协程是函数级别的程序，是可以通过程序员进行调用的。以下是协程特性的总结：

协程|属性
---|:--:
所需线程|单线程 (因为仅定义一个loop，所有event均在一个loop中)
编程方式|同步
实现效果|异步
是否需要锁机制|否
程序级别|函数级
实现机制|事件循环＋协程
总耗时|最耗时事件的时间
应用场景|IO密集型任务等

## Python异步IO之协程(二):使用asyncio的不同方法实现协程
https://blog.csdn.net/SL_World/article/details/86691747

在多个协程中的线性控制流很容易通过内置的关键词await来管理。使用asyncio模块中的方法可以实现更多复杂的结构，它可以并发地完成多个协程。

### asyncio.wait()
你可以将一个操作分成多个部分并分开执行，而wait(tasks)可以被用于中断任务集合(tasks)中的某个被事件循环轮询到的任务，直到该协程的其他后台操作完成才被唤醒。

In [1]:
import time
import asyncio
async def taskIO_1():
    print('开始运行IO任务1...')
    await asyncio.sleep(2)  # 假设该任务耗时2s
    print('IO任务1已完成，耗时2s')
    return taskIO_1.__name__
async def taskIO_2():
    print('开始运行IO任务2...')
    await asyncio.sleep(3)  # 假设该任务耗时3s
    print('IO任务2已完成，耗时3s')
    return taskIO_2.__name__
async def main(): # 调用方
    tasks = [taskIO_1(), taskIO_2()]  # 把所有任务添加到task中
    done,pending = await asyncio.wait(tasks) # 子生成器
    print(done)
    for r in done: # done和pending都是一个任务，所以返回结果需要逐个调用result()
        print('协程无序返回值：'+r.result())

if __name__ == '__main__':
    start = time.time()
    loop = asyncio.get_event_loop() # 创建一个事件循环对象loop
    try:
        loop.run_until_complete(main()) # 完成事件循环，直到最后一个任务结束
    finally:
        loop.close() # 结束事件循环
    print('所有IO任务总耗时%.5f秒' % float(time.time()-start))

开始运行IO任务2...
开始运行IO任务1...
IO任务1已完成，耗时2s
IO任务2已完成，耗时3s
{<Task finished coro=<taskIO_2() done, defined at <ipython-input-1-9d3ced649b08>:8> result='taskIO_2'>, <Task finished coro=<taskIO_1() done, defined at <ipython-input-1-9d3ced649b08>:3> result='taskIO_1'>}
协程无序返回值：taskIO_2
协程无序返回值：taskIO_1
所有IO任务总耗时3.00500秒


【解释】：wait()官方文档用法如下：

`done, pending = await asyncio.wait(aws)`

此处并发运行传入的aws(awaitable objects)，同时通过await返回一个包含(done, pending)的元组，done表示已完成的任务列表，pending表示未完成的任务列表。

注：
①只有当给wait()传入timeout参数时才有可能产生pending列表。
②通过wait()返回的结果集是按照事件循环中的任务完成顺序排列的，所以其往往和原始任务顺序不同。


##  asyncio 学习笔记
https://mozillazg.com/tag/asyncio.html

### asyncio 学习笔记：基本用法

#### 启动一个协程

启动一个协程的最简单的一个方法就是使用 asyncio.run_until_complete 函数：

In [1]:
import asyncio

async def coroutine():
    print('in coroutine')

event_loop = asyncio.get_event_loop() # 获取一个事件循环
try:
    print('starting coroutine')
    coro = coroutine() # Coroutine 对象
    print('entering event loop') 
    event_loop.run_until_complete(coro)  # 执行 Coroutine 对象
finally:
    print('closing event loop')
    event_loop.close()

starting coroutine
entering event loop
in coroutine
closing event loop


首先是获取一个事件循环，然后用 run_until_complete 执行 Coroutine 对象，当 Coroutine 执行完成并退出时， run_until_complete 也会随后退出。

#### 获取协程的返回值

run_until_complete 会把 Coroutine 的返回值当做自身的返回值返回给调用方

In [1]:
import asyncio

async def coroutine():
    print('in coroutine')
    return 'result'  # Coroutine 的返回值

event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(coroutine()) # 返回值返回给调用方
    print('it returned: {!r}'.format(return_value))
finally:
    event_loop.close()

in coroutine
it returned: 'result'


#### 链式调用协程
即在一个协程函数中调用另外的协程函数，同时还需要等待这个协程函数返回结果。

In [1]:
import asyncio

async def one():
    print('in one')
    return 'one'

async def two(arg):
    print('in two')
    return 'two with arg {}'.format(arg)

async def outer():
    print('in outer')
    print('waiting for one')
    result1 = await one()
    print('waiting for two')
    result2 = await two(result1)
    return result1, result2

event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('result value: {!r}'.format(return_value))
finally:
    event_loop.close()

in outer
waiting for one
in one
waiting for two
in two
result value: ('one', 'two with arg one')


可以直接使用 await 等待协程返回结果。

#### 使用生成器代替协程

在 Python 3.5 之前的 Python 3 版本中还没有 async/await 语法，我们可以使用 asyncio.coroutine 装饰器加 yield from 来实现同样的功能:

In [1]:
import asyncio


@asyncio.coroutine
def one():
    print('in one')
    return 'one'


@asyncio.coroutine
def two(arg):
    print('in two')
    return 'two with arg {}'.format(arg)


@asyncio.coroutine
def outer():
    print('in outer')
    print('waiting for one')
    result1 = yield from one()
    print('waiting for two')
    result2 = yield from two(result1)
    return result1, result2


event_loop = asyncio.get_event_loop()
try:
    return_value = event_loop.run_until_complete(outer())
    print('result value: {!r}'.format(return_value))
finally:
    event_loop.close()

in outer
waiting for one
in one
waiting for two
in two
result value: ('one', 'two with arg one')


## [Python并发编程之初识异步IO框架：asyncio](https://zhuanlan.zhihu.com/p/37346974)

asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。

有些同学，可能很疑惑，既然有了以生成器为基础的协程，我们直接使用yield 和 yield from 不就可以手动实现对IO的调度了吗？ 为何Python吃饱了没事干，老重复造轮子。

这个问题很好回答，就跟为什么会有Django，为什么会有Scrapy，是一个道理。他们都是框架，将很多很重复性高，复杂度高的工作，提前给你做好，这样你就可以专注于业务代码的研发。

你是不是也发现了，协程的知识点我已经掌握了，但是我还是不知道怎么用，如何使用，都说它可以实现并发，但是我还是不知道如何入手？那是因为，我们现在还缺少一个成熟的框架，帮助你完成那些复杂的动作。这个时候，ayncio就这么应运而生了。


### 如何定义/创建协程

#### 函数前面加上 async 关键字
只要在一个函数前面加上 async 关键字，这个函数对象是一个协程，通过isinstance函数，它确实是Coroutine类型。

In [1]:
from collections.abc import Coroutine

async def hello(name):
    print('Hello,',name)
    
if __name__ == '__main__':
    # 生成协程对象，并不会运行函数内的代码
    coroutine = hello('world')
    # 检查是否是协程 Coroutine 类型
    print(isinstance(coroutine, Coroutine))

True


前面说生成器是协程的基础，那我们是不是有办法，将一个生成器，直接变成协程使用呢。答案是有的。

#### 生成器函数头部用上 @asyncio.coroutine 装饰器

只要在一个生成器函数头部用上 @asyncio.coroutine 装饰器就能将这个函数对象，【标记】为协程对象。注意这里是【标记】，划重点。
实际上，它的本质还是一个生成器。标记后，它实际上已经可以当成协程使用。后面会介绍。

In [5]:
import asyncio
from collections.abc import Generator,Coroutine

@asyncio.coroutine
def hello():
    # 步调用asyncio.sleep(1)
    yield from asyncio.sleep(1)

if __name__ == '__main__':
    coroutine = hello()
    print(isinstance(coroutine, Generator))  
    print(isinstance(coroutine, Coroutine))  

True
False


c:\users\win7\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: coroutine 'hello' was never awaited
  # Remove the CWD from sys.path while we load stuff.


### asyncio的几个概念

在了解asyncio的使用方法前，首先有必要先介绍一下，这几个贯穿始终的概念。

- event_loop 事件循环：程序开启一个无限的循环，程序员会把一些函数（协程）注册到事件循环上。当满足事件发生的时候，调用相应的协程函数。
- coroutine 协程：协程对象，指一个使用async关键字定义的函数，它的调用不会立即执行函数，而是会返回一个协程对象。协程对象需要注册到事件循环，由事件循环调用。
- future 对象： 代表将来执行或没有执行的任务的结果。它和task上没有本质的区别
- task 任务：一个协程对象就是一个原生可以挂起的函数，任务则是对协程进一步封装，其中包含任务的各种状态。Task 对象是 Future 的子类，它将coroutine 和 Future 联系在一起，将 coroutine 封装成一个 Future 对象。
- async/await 关键字：python3.5 用于定义协程的关键字，async定义一个协程，await用于挂起阻塞的异步调用接口。其作用在一定程度上类似于yield。

这几个概念，干看可能很难以理解，没事，往下看实例，然后再回来，我相信你一定能够理解。

### 学习协程是如何工作的

协程完整的工作流程是这样的

- 定义/创建协程对象
- 定义事件循环对象容器
- 将协程转为task任务
- 将task任务扔进事件循环对象中触发

In [6]:
import asyncio

async def hello(name):
    print('Hello,',name)

# 定义协程对象
coroutine = hello('World')

# 定义事件循环对象容器
loop = asyncio.get_event_loop()
# task = asyncio.ensure_future(coroutine)

# 将协程转为task任务
task = loop.create_task(coroutine)

# 将task任务扔进事件循环对象中并触发
loop.run_until_complete(task)

Hello, World


### await与yield对比

前面我们说，await用于挂起阻塞的异步调用接口。其作用在一定程度上类似于yield。注意这里是，一定程度上，意思是效果上一样（都能实现暂停的效果），但是功能上却不兼容。**就是你不能在生成器中使用await，也不能在async 定义的协程中使用yield。**

